<a href="https://colab.research.google.com/github/peterij/TM10007_Diagnosing_heart_disease/blob/Peter/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TM10007 Machine Learning**

In [ ]:
!git clone https://github.com/jveenland/tm10007_ml.git

import zipfile
import os
import pandas as pd

with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')

data = pd.read_csv('/content/tm10007_ml/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

In [ ]:
# import function
from sklearn import model_selection


In [ ]:
data['label'].sum()

In [ ]:
## test test test test


In [ ]:
ECG_NON_NORMAL

In [ ]:





x_train, x_test = model_selection.train_test_split(data, test_size=0.2)
#print(x_train)





print(f'The number of samples: {len(x_train.index)}')
print(f'The number of columns: {len(x_train.columns)}')
# Stratified split met vergelijkbare ratio 
